<center>

# Constructing Pauli Twirling Sets from Arbitrary Error Channels
</center>


Note: All these paragraphs are stolen from the paper and need editing.

"Twirling is a technique widely used for converting arbitrary noise channels into Pauli channels in error threshold estimations of quantum error correction codes. It is vitally useful both in real experiments and in classical quantum simulations. Minimising the size of the twirling gate set increases the efficiency of simulations and in experiments it might reduce both the number of runs required and the circuit depth (and hence the error burden). Conventional twirling uses the full set of Pauli gates as the set of twirling gates. This article provides a theoretical background for Pauli twirling and a way to construct a twirling gate set with a number of members comparable to the size of the Pauli basis of the given error channel, which is usually much smaller than the full set of Pauli gates. We also show that twirling is equivalent to stabiliser measurements with discarded measurement results, which enables us to further reduce the size of the twirling gate set."

<center>

## Introduction

</center>


Twirling is a technique that has been long established in the quantum information literature. It was first used for mapping a diverse range of states into a canonical form in entanglement purification [1, 2]. Then it ap- peared again as an integral part in randomised bench- marking [3, 4] and was also used to reduce the num- ber of experimental runs needed in quantum process tomography [5, 6], both are critical in benchmarking the performance of quantum systems, especially “Noisy Intermediate-Scale Quantum” (NISQ) systems [7]. More recently, twirling was used as means to boost the perfor- mance of NISQ through error mitigations [8–11] in which it enables a controlled increase of the gate error rates for error extrapolations. In this article, twirling is discussed as a technique for simulating noise and the impact of the noise on the performance of quantum error correction codes [12].

The Gottesman-Knill theorem [13, 14] states that any quantum circuits involving only Clifford gates can be per- fectly simulated in polynomial time on a classical com- puter. One important example is the circuits used to im- plement quantum error correction codes. For each code, there exists an error threshold of the circuit components below which the computational error can be made arbi- trarily small by scaling up the code. As we try to obtain the error thresholds of the codes, we often need to in- troduce various forms of noise into the circuits based on the underlying physical implementations. This noise can be viewed as extra probabilistic gates on top of the per- fect Clifford gates. However, the fact that this noise can be non-Clifford means that the circuits cannot be simu- lated efficiently classically, i.e. numerically determining the threshold becomes intractable.

This can be solved by twirling. Twirling means that every time we run the circuit, we conjugate the noise with an gate randomly chosen from a set of gates called the twirling set. By choosing the twirling set to be the full set of Pauli operators, we can convert any noise channel into a Pauli channel whose noise elements correspond to the Pauli basis of the original noise [15]. Such Pauli chan- nel approximation has been shown to be effective in error threshold estimation by Geller et al. [12] and Guti ́errez et al. [16], which justify its usage in error threshold simula- tion across various architectures [17–20].

<center>

## Purpose of the Project

</center>

In this article we will focus on Pauli twirling, whose twirling set is a subset of Pauli gates, with the goal of converting a given noise channel into a Pauli channel. For such a goal, twirling over the full set of Pauli operators is not always optimal. If we want to apply twirling in quan- tum simulations or real experiments, a twirling set with a smaller size means a lower number of simulations or experiments may be needed to get the full statistical re- sult. Moreover, a smaller twirling set allows us to choose twirling gates that have higher fidelities and/or act on fewer qubits. This will reduce the number of errors we introduce into the system due to twirling.

In this article, we will introduce a way to exploit the symmetries in the noise channel to reduce the size of the Pauli twirling set needed for the channel. The paper is or- ganised as follows. In Section II, we first introduce some essential concepts for our analysis. In Section III, we in- troduce the theory of Pauli twirling, in which we obtain the requirement on the twirling set. In Section IV, we show a way to construct a twirling set that satisfied the conditions that we laid out. This is followed by two exam- ples. In Section V, we discuss how to use stabiliser mea- surements to further reduce the size of our twirling set. Lastly, Section VI provides a summary of our results and some possible future directions. The mathematical justi- fication for our method of construction of the twirling set is described in the appendices, which forms an essential part of the paper.

<center>

## Constructing the Twirling Set
<center>

### Building Commutator and Generator Tables to Compute $W$

The first step is creating the $n$-qubit Pauli operator $G$ defined as,
<center>

$G = \left\{{I,X,Y,Z}\right\}^{\otimes n}$

</center>

In [13]:
import numpy as np
from itertools import product

# Define Pauli matrices and their names
pauli_matrices = {
    'I': np.array([[1, 0], [0, 1]]),
    'X': np.array([[0, 1], [1, 0]]),
    'Y': np.array([[0, -1j], [1j, 0]]),
    'Z': np.array([[1, 0], [0, -1]])
}

# Generate tensor products for multi-qubit Pauli operators
def generate_multi_qubit_paulis(n):
    keys = list(pauli_matrices.keys())
    ops = list(pauli_matrices.values())
    names = []
    matrices = []

    for prod in product(keys, repeat=n):
        name = ''.join(prod)
        matrix = pauli_matrices[prod[0]]
        for k in range(1, n):
            matrix = np.kron(matrix, pauli_matrices[prod[k]])
        names.append(name)
        matrices.append(matrix)

    return names, matrices

# Define commutator function ζ(gi, gj)
def commutator(gi, gj):
    comm = gi @ gj - gj @ gi
    return 1 if np.allclose(comm, 0) else -1

# Generate commutator table
def generate_commutator_table(matrices, names):
    n = len(matrices)
    table = np.zeros((n, n), dtype=int)
    
    for i in range(n):
        for j in range(n):
            table[i, j] = commutator(matrices[i], matrices[j])
    
    # Print with names
    print("\t" + "\t".join(names))
    for i in range(n):
        print(names[i] + "\t" + "\t".join(map(str, table[i])))

#### Commutator Table

In [14]:
# Example usage for 2-qubit Pauli operators
names, matrices = generate_multi_qubit_paulis(2)
generate_commutator_table(matrices, names)

	II	IX	IY	IZ	XI	XX	XY	XZ	YI	YX	YY	YZ	ZI	ZX	ZY	ZZ
II	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1
IX	1	1	-1	-1	1	1	-1	-1	1	1	-1	-1	1	1	-1	-1
IY	1	-1	1	-1	1	-1	1	-1	1	-1	1	-1	1	-1	1	-1
IZ	1	-1	-1	1	1	-1	-1	1	1	-1	-1	1	1	-1	-1	1
XI	1	1	1	1	1	1	1	1	-1	-1	-1	-1	-1	-1	-1	-1
XX	1	1	-1	-1	1	1	-1	-1	-1	-1	1	1	-1	-1	1	1
XY	1	-1	1	-1	1	-1	1	-1	-1	1	-1	1	-1	1	-1	1
XZ	1	-1	-1	1	1	-1	-1	1	-1	1	1	-1	-1	1	1	-1
YI	1	1	1	1	-1	-1	-1	-1	1	1	1	1	-1	-1	-1	-1
YX	1	1	-1	-1	-1	-1	1	1	1	1	-1	-1	-1	-1	1	1
YY	1	-1	1	-1	-1	1	-1	1	1	-1	1	-1	-1	1	-1	1
YZ	1	-1	-1	1	-1	1	1	-1	1	-1	-1	1	-1	1	1	-1
ZI	1	1	1	1	-1	-1	-1	-1	-1	-1	-1	-1	1	1	1	1
ZX	1	1	-1	-1	-1	-1	1	1	-1	-1	1	1	1	1	-1	-1
ZY	1	-1	1	-1	-1	1	-1	1	-1	1	-1	1	1	-1	1	-1
ZZ	1	-1	-1	1	-1	1	1	-1	-1	1	1	-1	1	-1	-1	1


#### Generator Table

In [15]:
# Example: Generator table for selected operators
selected_names = ['XI', 'YX', 'IZ', 'XX']
selected_matrices = [matrices[names.index('XI')], matrices[names.index('YX')], matrices[names.index('IZ')], matrices[names.index('XX')]]

generate_commutator_table(selected_matrices, selected_names)

	XI	YX	IZ	XX
XI	1	-1	1	1
YX	-1	1	-1	-1
IZ	1	-1	1	-1
XX	1	-1	-1	1


### Finding $W$ Using Guassian Elimination

In [16]:
import numpy as np
from useful_classes import Paulis
from typing import List
from sympy import Matrix





def basis(e: np.ndarray, p: Paulis) -> List[np.ndarray]:
    """
    Finds the Pauli basis elements from that contribute to the noise operator `e`.

    Args:
        e (np.ndarray): The noise operator matrix.
        p (Paulis): An object containing a collection of Pauli matrices in `p.multi_p`.

    Returns:
        List[np.ndarray]: A list of Pauli basis elements for which Tr(g @ e) != 0.
    """
    return [g for g in p.multi_p if np.trace(g @ e) != 0]

def find_generating_set(V):
    """
    Finds the minimal generating set Ṽ for the given set V of Pauli operators.

    Args:
        V (list[str]): List of Pauli operators as strings (e.g., ["IX", "IZ", "YX"]).

    Returns:
        list[str]: Minimal independent generating set Ṽ.
    """
    # Map Pauli operators to binary vectors
    binary_vectors = [pauli_to_binary(v) for v in V]

    # Perform Gaussian elimination (mod 2)
    M = Matrix(binary_vectors)
    M_reduced, _ = M.rref()  # Reduced row echelon form

    # Extract independent operators
    independent_indices = [i for i, row in enumerate(M_reduced.tolist()) if any(row)]
    return [V[i] for i in independent_indices]

def pauli_to_binary(pauli):
    """
    Converts a Pauli operator to its binary vector representation.

    Args:
        pauli (str): Pauli operator (e.g., "IX", "IZ").

    Returns:
        list[int]: Binary vector representation.
    """
    mapping = {'I': (0, 0), 'X': (1, 0), 'Z': (0, 1), 'Y': (1, 1)}
    return [bit for char in pauli for bit in mapping[char]]


def main():
    p = Paulis(2)
    print(find_generating_set(['XX', "YY", "ZI", 'II', "ZZ"]))
    print(find_generating_set(['IX', 'IZ', 'YX', 'ZX', 'YY']))

if __name__ == "__main__":
    main()


['XX', 'YY', 'ZI']
['IX', 'IZ', 'YX', 'ZX']


# Detailed Guide on Importing and Using The Quantum Twirling Code in Another Project

This guide will walk you through the process of encapsulating our quantum twirling code into a single function within a module, and then importing and using that function in another code project. The goal is to make your code modular, reusable, and easy to integrate.


## Step 1: Organize Your Code into a Module

First, you need to organize your existing code into a Python module. A module is simply a Python file (`.py` extension) that can be imported and used in other Python scripts.

### Actions:

- **Create a new Python file**: Let's name it `twirling.py`.
- **Ensure all necessary functions and imports are included**: This includes the functions you've defined (`_find_generating_set`, `_pauli_to_binary`, etc.) and any class imports like `Paulis` and `Q_states`.

---

## Step 2: Create a Wrapper Function

To make your code easily callable, encapsulate the main functionality into a single wrapper function. This function will take necessary inputs and internally call the other functions.

### Define the Wrapper Function

```python
def run_twirling_simulation(error_model: dict[str, dict]) -> dict[str, float]:
    """
    Executes the twirling simulation given an error model.
    
    Args:
        error_model (dict[str, dict]): The error model with state-specific errors.
    
    Returns:
        dict[str, float]: Twirling results with Pauli errors and their probabilities.
    """
    num_qubits = len(next(iter(error_model)))
    P = Paulis(num_qubits)
    K = get_kraus_operators(error_model, P)
    validate_kraus_operators(K)
    twirling_results = twirl(error_model)
    return twirling_results
```

### What the Wrapper Function Does:

- **Determines the number of qubits** based on the error model.
- **Initializes necessary objects** (e.g., `Paulis`).
- **Generates Kraus operators** using `get_kraus_operators`.
- **Validates the Kraus operators** with `validate_kraus_operators`.
- **Performs the twirling** using `twirl`.
- **Returns the results** as a dictionary.

---

## Step 3: Save the Module

Make sure all your code, including the wrapper function and necessary imports, is saved in `twirling.py`.

### Example Structure of `twirling.py`:

```python
import numpy as np
from useful_classes import Paulis, Q_states
from sympy import Matrix

# ... [all other functions up here] ...

def run_twirling_simulation(error_model: dict[str, dict]) -> dict[str, float]:
    # [Wrapper function code]
```

---

## Step 4: Import and Use the Module in Another Project

Now that your module is ready, you can import it into another project and call the wrapper function.

### Actions:

- **Create a new Python script**: For example, `main_script.py`.
- **Import the module**: Use the `import` statement to bring in your module.
- **Call the wrapper function**: Pass the appropriate arguments.

---

## Step 5: Example Usage

### main_script.py

```python
import twirling  # Import your module

def main():
    # Define your error model
    error_model = {
        "11": {"XX": 0.1},
        "10": {"XI": 0.3},
        "01": {"IZ": 0.2}
    }
    
    # Call the wrapper function
    results = twirling.run_twirling_simulation(error_model)
    
    # Display the results
    print("Twirling Results:")
    for pauli_error, probability in results.items():
        print(f"{pauli_error}: {probability:.6f}")

if __name__ == "__main__":
    main()
```

### Explanation:

- **Import Statement**: `import twirling` brings in the module you've created.
- **Define Error Model**: The `error_model` variable holds the state-specific errors.
- **Call Function**: `results = twirling.run_twirling_simulation(error_model)` runs the simulation.
- **Output**: The results are printed to the console.

---

## Step 6: Full Code Listings

### twirling.py

```python
import numpy as np
from useful_classes import Paulis, Q_states
from sympy import Matrix

# ... [all other functions up here] ...

def run_twirling_simulation(error_model: dict[str, dict]) -> dict[str, float]:
    num_qubits = len(next(iter(error_model)))
    P = Paulis(num_qubits)
    K = get_kraus_operators(error_model, P)
    validate_kraus_operators(K)
    twirling_results = twirl(error_model)
    return twirling_results
```

### main_script.py

```python
import twirling

def main():
    error_model = {
        "11": {"XX": 0.1},
        "10": {"XI": 0.3},
        "01": {"IZ": 0.2}
    }
    
    results = twirling.run_twirling_simulation(error_model)
    
    print("Twirling Results:")
    for pauli_error, probability in results.items():
        print(f"{pauli_error}: {probability:.6f}")

if __name__ == "__main__":
    main()
```

---

## Step 7: Notes and Tips

### Ensure Dependencies are Met

- **useful_classes Module**: Make sure the `Paulis` and `Q_states` classes from `useful_classes` are accessible. They should either be in the same directory or installed as a package.
- **Sympy and Numpy**: Ensure `sympy` and `numpy` are installed in your Python environment.

### Adjust the Error Model as Needed

- The error model is a dictionary where:
  - **Keys**: Strings representing quantum states (e.g., `"11"`).
  - **Values**: Dictionaries mapping Pauli errors to their probabilities (e.g., `{"XX": 0.1}`).
- **Probabilities**: Make sure the probabilities are valid (i.e., between 0 and 1) and that they sum appropriately for each state.

### Testing with Different Error Models

Add or modify the `error_model` in `main_script.py` to test different scenarios.

#### Example:

```python
def main():
    error_model = {
        "00": {"II": 0.9, "XX": 0.1},
        "01": {"YY": 0.2, "ZZ": 0.8},
        "10": {"XI": 0.5, "IX": 0.5},
        "11": {"XX": 0.4, "YY": 0.6}
    }
    
    results = twirling.run_twirling_simulation(error_model)
    
    print("Twirling Results:")
    for pauli_error, probability in results.items():
        print(f"{pauli_error}: {probability:.6f}")
```

### Error Handling

- The `validate_kraus_operators` function checks if the Kraus operators satisfy necessary quantum channel properties.
- If there's an issue with the error model or the operators, an exception will be raised. Be prepared to handle exceptions or adjust the error model accordingly.

---

## Conclusion

By following these steps, you've successfully encapsulated your quantum twirling code into a module and learned how to import and use it in another project. This approach enhances code reusability and maintainability, allowing you to integrate complex functionality with ease.

---

## Additional Resources

- **Python Modules and Packages**: [Official Documentation](https://docs.python.org/3/tutorial/modules.html)
- **Importing Modules**: [Python Import System](https://docs.python.org/3/reference/import.html)
- **Exception Handling**: [Python Exceptions](https://docs.python.org/3/tutorial/errors.html)

---




